In [14]:
!cat container/Dockerfile

FROM nvidia/cuda:9.0-base-ubuntu16.04

MAINTAINER Amazon AI

ARG framework_installable=tensorflow-1.9.0-cp27-cp27mu-manylinux1_x86_64.whl
ARG framework_support_installable=sagemaker_tensorflow_container-1.0.0.tar.gz

RUN apt-get update && apt-get install -y --no-install-recommends \
        build-essential \
        cuda-command-line-tools-9-0 \
        cuda-cublas-dev-9-0 \
        cuda-cudart-dev-9-0 \
        cuda-cufft-dev-9-0 \
        cuda-curand-dev-9-0 \
        cuda-cusolver-dev-9-0 \
        cuda-cusparse-dev-9-0 \
        curl \
        git \
        libcudnn7=7.1.4.18-1+cuda9.0 \
        libcudnn7-dev=7.1.4.18-1+cuda9.0 \
        libcurl3-dev \
        libfreetype6-dev \
        libpng12-dev \
        libzmq3-dev \
        pkg-config \
        python-dev \
        rsync \
        software-properties-common \
        unzip \
        zip \
        zlib1g-dev \
        wget \
        vim \
        nginx \
        iputils-ping \
        && \
    rm -rf /var/lib/apt/lists/* && \

### Building and registering the container

The following shell code shows how to build the container image using `docker build` and push the container image to ECR using `docker push`. This code is also available as the shell script `container/build-and-push.sh`, which you can run as `build-and-push.sh decision_trees_sample` to build the image `decision_trees_sample`. 

This code looks for an ECR repository in the account you're using and the current default region (if you're using a SageMaker notebook instance, this will be the region where the notebook instance was created). If the repository doesn't exist, the script will create it.

In [16]:
%%sh

# The name of our algorithm
algorithm_name=keras-test

cd container

chmod +x keras-test/train
chmod +x keras-test/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.

aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

Login Succeeded

Step 1/43 : FROM nvidia/cuda:9.0-base-ubuntu16.04
 ---> 9dcd7cd95db6
Step 2/43 : MAINTAINER Amazon AI
 ---> Using cache
 ---> 972f6d23a93d
Step 3/43 : ARG framework_installable=tensorflow-1.9.0-cp27-cp27mu-manylinux1_x86_64.whl
 ---> Using cache
 ---> 78aec2c5b4a5
Step 4/43 : ARG framework_support_installable=sagemaker_tensorflow_container-1.0.0.tar.gz
 ---> Using cache
 ---> e0da941810dd
Step 5/43 : RUN apt-get update && apt-get install -y --no-install-recommends         build-essential         cuda-command-line-tools-9-0         cuda-cublas-dev-9-0         cuda-cudart-dev-9-0         cuda-cufft-dev-9-0         cuda-curand-dev-9-0         cuda-cusolver-dev-9-0         cuda-cusparse-dev-9-0         curl         git         libcudnn7=7.1.4.18-1+cuda9.0         libcudnn7-dev=7.1.4.18-1+cuda9.0         libcurl3-dev         libfreetype6-dev         libpng12-dev         libzmq3-dev         pkg-config         python-dev         rsync         software-properties-common       

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



## Testing your algorithm on your local machine or on an Amazon SageMaker notebook instance

While you're first packaging an algorithm use with Amazon SageMaker, you probably want to test it yourself to make sure it's working right. In the directory `container/local_test`, there is a framework for doing this. It includes three shell scripts for running and using the container and a directory structure that mimics the one outlined above.

The scripts are:

* `train_local.sh`: Run this with the name of the image and it will run training on the local tree. You'll want to modify the directory `test_dir/input/data/...` to be set up with the correct channels and data for your algorithm. Also, you'll want to modify the file `input/config/hyperparameters.json` to have the hyperparameter settings that you want to test (as strings).
* `serve_local.sh`: Run this with the name of the image once you've trained the model and it should serve the model. It will run and wait for requests. Simply use the keyboard interrupt to stop it.
* `predict.sh`: Run this with the name of a payload file and (optionally) the HTTP content type you want. The content type will default to `text/csv`. For example, you can run `$ ./predict.sh payload.csv text/csv`.

The directories as shipped are set up to test the decision trees sample algorithm presented here.

# Part 2: Training and Hosting your Algorithm in Amazon SageMaker

Once you have your container packaged, you can use it to train and serve models. Let's do that with the algorithm we made above.

## Set up the environment

Here we specify a bucket to use and the role that will be used for working with SageMaker.

In [3]:
# S3 prefix
prefix = 'boxing'

# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

## Create the session

The session remembers our connection parameters to SageMaker. We'll use it to perform all of our SageMaker operations.

In [4]:
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

## Upload the data for training

When training large models with huge amounts of data, you'll typically use big data tools, like Amazon Athena, AWS Glue, or Amazon EMR, to create your data in S3. For the purposes of this example, we're using some the classic [Iris dataset](https://en.wikipedia.org/wiki/Iris_flower_data_set), which we have included. 

We can use use the tools provided by the SageMaker Python SDK to upload the data to a default bucket. 

In [5]:
WORK_DIRECTORY = 'data'

data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)

INFO:sagemaker:Created S3 bucket: sagemaker-us-east-1-452432741922


## Create an estimator and fit the model

In order to use SageMaker to fit our algorithm, we'll create an `Estimator` that defines how to use the container to train. This includes the configuration we need to invoke SageMaker training:

* The __container name__. This is constructed as in the shell commands above.
* The __role__. As defined above.
* The __instance count__ which is the number of machines to use for training.
* The __instance type__ which is the type of machine to use for training.
* The __output path__ determines where the model artifact will be written.
* The __session__ is the SageMaker session object that we defined above.

Then we use fit() on the estimator to train against the data that we uploaded above.

In [6]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/keras-test:latest'.format(account, region)

model = sage.estimator.Estimator(image,
                       role, 1, 'ml.p2.16xlarge',
                       output_path="s3://{}/output".format(sess.default_bucket()),
                       sagemaker_session=sess)

model.fit(data_location)

INFO:sagemaker:Creating training-job with name: keras-test-2019-04-10-15-02-32-597


2019-04-10 15:02:32 Starting - Starting the training job...
2019-04-10 15:02:35 Starting - Launching requested ML instances.........
2019-04-10 15:04:16 Starting - Preparing the instances for training............
2019-04-10 15:06:29 Downloading - Downloading input data
2019-04-10 15:06:29 Training - Downloading the training image.........
2019-04-10 15:07:48 Uploading - Uploading generated training model...
2019-04-10 15:08:24 Completed - Training job completed
..
Billable seconds: 123


In [10]:
account = '452432741922'

region = 'us-east-1'

image = '452432741922.dkr.ecr.us-east-1.amazonaws.com/keras-test:latest'

'452432741922'

## Deploy the model

Deploying the model to SageMaker hosting just requires a `deploy` call on the fitted model. This call takes an instance count, instance type, and optionally serializer and deserializer functions. These are used when the resulting predictor is created on the endpoint.

In [7]:
from sagemaker.predictor import csv_serializer
predictor = model.deploy(1, 'ml.p3.8xlarge', serializer=csv_serializer)

INFO:sagemaker:Creating model with name: keras-test-2019-04-10-15-08-48-782
INFO:sagemaker:Creating endpoint with name keras-test-2019-04-10-15-02-32-597


----------------------------------------------------------------------------------------!

In [8]:
#!pip install opencv-python

In [ ]:
import requests
import json
import cv2
img = cv2.imread('test.jpeg')

In [ ]:
_, img_encoded = cv2.imencode('.jpg', img)

In [ ]:
content_type = 'image/jpeg'
headers = {'content-type': content_type}

In [ ]:
img_encoded.shape

In [ ]:
import boto3

client = boto3.client('sagemaker-runtime')

endpoint_name = "keras-test-2019-04-05-16-27-28-388"                                       # Your endpoint name.
content_type = "image/jpeg"                                        # The MIME type of the input data in the request body.
payload = img_encoded.tostring()                                             # Payload for inference.
 

In [ ]:
%%timeit
response = client.invoke_endpoint(
    EndpointName=endpoint_name, 
    ContentType=content_type,
    Body=payload
    )

print(response)                         # If model receives and updates the custom_attributes header 
                                   

In [ ]:
response['Body'].read()

## Optional cleanup

When you're done with the endpoint, you'll want to clean it up.

In [ ]:
sess.delete_endpoint(predictor.endpoint)